It seems like we discovered genres from the first LDA we performed. Can we discover more topics from within those genres?

We can try clustering on the results of our LDA first

In [21]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.decomposition import NMF
from nltk.corpus import stopwords
from unidecode import unidecode
from collections import Counter
import string

In [30]:
path = '../../data/lyrics/data/subset_bow_2.csv'
df = pd.read_csv(path,index_col=0)

In [31]:
to_drop = []
for ind in df.index:
    if df.loc[ind].sum()==0:
        to_drop.append(ind)
df = df.drop(to_drop)

In [34]:
stop_words = stopwords.words('english') + stopwords.words('spanish') + stopwords.words('german') + stopwords.words('french')
col_keep = []
for word in df.columns:
    if word not in stop_words:
        col_keep.append(word)
df = df[col_keep]

/Users/christopherliu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [35]:
df.shape

(2350, 4645)

In [64]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
n = 18
lda = LDA(n_topics=n,max_iter=30,learning_method='online')
X = lda.fit_transform(df)

In [65]:
components = lda.components_
songs_mask = np.argsort(X,axis=0).T
songs_mask = np.array([mask[::-1] for mask in songs_mask])[:,:5]
imp_3track_ids = np.array(df.index)[songs_mask]

mask=np.array([row[::-1] for row in np.argsort(components,axis=1)])[:,:100]
imp_words = np.array(df.columns)[mask]


conn = sqlite3.connect('../../data/MillionSongSubset/AdditionalFiles/subset_track_metadata.db')
q = "SELECT title, artist_name FROM songs WHERE track_id = '{}';"
# q2 = "SELECT title, artist_name FROM songs WHERE track_id = 'TRAWVWS128F42ADCD1';"
# print q.format(imp_track_ids[0])
for track_ids, topic in zip(imp_3track_ids,imp_words):
    print topic[:10]
    formatted = '\t"{}" by {}'
    for track_id in track_ids:
        song,artist= conn.execute(q.format(track_id)).fetchone()
        print formatted.format(unidecode(song), unidecode(artist), topic[0:5])
conn.close()

['quiero' 'si' 'amor' 'mas' 'solo' 'corazon' 'vida' 'nunca' 'ser' 'vez']
	"Angelito (Directo 09)" by Carlos Baute
	"Hablando A Un Cristal" by Cafe Quijano
	"Celos" by Obie Bermudez
	"Tu Robaste Mi Corazon" by Selena
	"Estoy Contigo" by Selena
['hey' 'hot' 'roll' 'wish' 'wild' '&' 'gonna' 'push' 'got' 'everybodi']
	"You Gotta Be Movin'" by Corona
	"Abagui Alobi" by Patience Dabany
	"Hot Wire [Live Version]" by Kix
	"Here I Am" by Phantom Planet
	"Stairway To Cleveland" by Jefferson Starship
['like' 'get' 'got' 'know' 'caus' 'make' 'wanna' 'want' 'see' 'say']
	"Get On Top (Album Version)" by Red Hot Chili Peppers
	"Call Me" by Mint Condition
	"Miss Elaine" by RUN-DMC
	"I'm A Chicken Head (Skit)" by Cam'Ron
	"School" by Nirvana
['oh' 'yeah' 'danc' 'ooh' 'ah' 'get' 'take' 'break' 'sexi' 'shake']
	"Nordmark" by YGGDRASIL
	"Contagious" by Y&T
	"Pianolude" by Usher
	"Sfm" by Basement Jaxx
	"This Is How A Heart Breaks" by Joe McBride
['g' 'denk' 'ja' 'immer' 'mehr' 'fur' 'lover' 'rap' 'drum' '

In [36]:
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters=8)

In [38]:
kmean.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [39]:
kmean.predict(X)

array([4, 3, 3, ..., 3, 1, 1], dtype=int32)

In [40]:
Counter(kmean.predict(X))

Counter({0: 356, 1: 441, 2: 171, 3: 811, 4: 191, 5: 82, 6: 45, 7: 253})

In [46]:
np.argsort(kmean.cluster_centers_,axis=1)

array([[10, 15,  3,  0, 13,  2, 11, 14,  4,  1, 17,  7, 12, 16,  5,  8,  6,
         9],
       [ 0, 13, 10, 14,  2,  3, 11, 12,  4,  8,  1,  7, 15, 17,  9,  6,  5,
        16],
       [ 7, 14,  6, 12,  1,  0,  9, 16,  5,  4,  2, 11,  8, 13, 17, 15, 10,
         3],
       [15, 11,  3, 10,  0, 13,  2, 14,  4,  1, 17,  7, 12,  5,  8, 16,  9,
         6],
       [15, 11,  3,  0, 10, 13,  2, 17,  4,  5,  7, 16,  1, 14,  9,  8,  6,
        12],
       [ 2, 16, 11, 14,  8,  7,  1,  5,  3, 17,  6, 15, 13,  0,  9, 12,  4,
        10],
       [ 2, 11,  3,  7,  0, 16,  1,  8, 14,  6, 10, 17,  4,  5,  9, 12, 15,
        13],
       [11,  3, 15, 13, 10,  0,  2, 14,  1, 17,  7,  4, 12,  5, 16,  9,  6,
         8]])

## Do some other stuff

In [66]:
Counter(np.argmax(X,axis=1))

Counter({0: 176,
         1: 13,
         2: 474,
         3: 19,
         4: 41,
         5: 147,
         6: 36,
         7: 1061,
         8: 100,
         9: 8,
         10: 26,
         11: 109,
         12: 11,
         13: 14,
         14: 11,
         15: 4,
         16: 18,
         17: 82})

In [67]:
spanish_topic = np.where(np.argmax(X,axis=1)==3)
feelings_topic = np.where(np.argmax(X,axis=1)==6)
feelings_topic

(array([  24,   88,  124,  227,  228,  239,  397,  423,  425,  449,  609,
         615,  618,  650,  769,  785,  844,  860,  874,  892, 1036, 1084,
        1117, 1138, 1173, 1331, 1438, 1449, 1465, 1485, 1489, 1531, 1660,
        2077, 2196, 2317]),)

In [70]:
lda2 = LDA(n_topics=8,max_iter=30,learning_method='online')
X2 = lda2.fit_transform(df.iloc[feelings_topic])

In [71]:
components2 = lda2.components_
songs_mask = np.argsort(X2,axis=0).T
songs_mask = np.array([mask[::-1] for mask in songs_mask])[:,:5]
imp_3track_ids = np.array(df.index)[songs_mask]

mask=np.array([row[::-1] for row in np.argsort(components2,axis=1)])[:,:100]
imp_words = np.array(df.columns)[mask]


conn = sqlite3.connect('../../data/MillionSongSubset/AdditionalFiles/subset_track_metadata.db')
q = "SELECT title, artist_name FROM songs WHERE track_id = '{}';"
# q2 = "SELECT title, artist_name FROM songs WHERE track_id = 'TRAWVWS128F42ADCD1';"
# print q.format(imp_track_ids[0])
for track_ids, topic in zip(imp_3track_ids,imp_words):
    print topic[:10]
    formatted = '\t"{}" by {}'
    for track_id in track_ids:
        song,artist= conn.execute(q.format(track_id)).fetchone()
        print formatted.format(unidecode(song), unidecode(artist), topic[0:5])
conn.close()

['whoa' 'oh' 'left' 'gonna' 'fantasi' 'well' 'take' 'jesus' 'say' 'care']
	"T-Bone" by T-Bone
	"Spawn (Again)" by Silverchair
	"The Majesty Of Rock" by Spinal Tap
	"Obvious ( LP Version )" by Jane's Addiction
	"Life Force" by Vanessa Daou
['goin' 'gone' 'rock' 'sound' 'tear' 'round' 'go' 'miss' 'yeah' 'babi']
	"Rebel Music" by Wyclef Jean featuring Prodigy of Mobb Deep
	"Oh My God" by Sticky Fingaz
	"Do Ya Thing (Remix) (Explicit Album Version)" by Cam'Ron
	"From Tense To Loose To Slack" by The Keys
	"The Dynamo Of Volition [From An All Night Session]" by Jason Mraz
['song' 'love' 'action' 'revolut' 'gonna' 'time' 'live' 'comin' 'go' 'big']
	"Everyone's At It" by Lily Allen
	"Fertile Crescent (bonus track)" by Bad Religion
	"Zig-Zag Walk" by Foghat
	"Rebel Music" by Wyclef Jean featuring Prodigy of Mobb Deep
	"Oh My God" by Sticky Fingaz
['chain' 'ah' 'lazi' 'oh' 'whoa' 'far' 'miser' 'got' 'king' 'ca']
	"Oh My God" by Sticky Fingaz
	"Life Force" by Vanessa Daou
	"Till We Ain't Stranger